Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 11.8 MB 2.1 MB/s 
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.1-cp37-cp37m-linux_x86_64.whl size=1633966 sha256=ae9d7a11d8905acf1b90d55a6e15abcad8f30909dc10b051e1f088014b7f83c6
  Stored in directory: /root/.cache/pip/wheels/76/44/74/b498c42be47b2406bd27994e16c5188e337c657025ab400c1c
Successfully built scikit-surprise


In [ ]:
import requests
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from google.cloud import bigquery

In [ ]:
# Authenticate google cloud platform
# from google.colab import auth
# auth.authenticate_user()
# print('Authenticated')

Authenticated


In [ ]:
project_id = 'listenbrainzrecommender'
client = bigquery.Client(project=project_id)

Even with an expanded ram memory to 25gb, we're still unable to load in the selected fields.
After some more exploration, we found that the dataset contains data from 2015 to 2018.

We'll limit the data to 2016 onwards to try and get the data loaded.

In [ ]:
# get all plays >= year 2016
big_plays = client.query('''
SELECT
  user_name, artist_name, track_name
FROM
  `listenbrainz.listenbrainz.listen`
WHERE
  listened_at > '2016-01-01 00:00:00 UTC'
''').to_dataframe()

In [ ]:
big_plays.head()

,user_name,artist_name,track_name
0,Skeebadoo,Katy B X Mr. Mitch,Heavy
1,Skeebadoo,These Hidden Hands,SZ31X71
2,Skeebadoo,Esben and the Witch,Marking the Heart of a Serpent
3,Skeebadoo,death's dynamic shroud.wmv,Loving Is Easy
4,Svarthjelm,Fuzz,What's In My Head?


In [ ]:
big_plays.describe()

,user_name,artist_name,track_name
count,37775028,37775028,37775028
unique,2644,561703,3293454
top,phonebox,Radiohead,Intro
freq,325414,157474,33273


In [ ]:
# Save df to csv on google drive so we don't have to query to get it again
# The query takes a long time(1.5 hrs)
from google.colab import drive

drive.mount('/drive')
big_plays.to_csv('/drive/My Drive/Colab Notebooks/full_data.csv')

Drive already mounted at /drive; to attempt to forcibly remount, call drive.mount("/drive", force_remount=True).


We will use this as our main dataset going forward. I'm content with 37 million records to use.

In [ ]:
# create df for only songs, to create unique id's for songs
# unique id will be index
song_df = big_plays[['artist_name', 'track_name']]
len(song_df)

37775028

In [ ]:
len(song_df_cl)

4660364

In [ ]:
# drop duplicate songs
song_df_cl = song_df.drop_duplicates()
len(song_df_cl)

4660364

In [ ]:
song_df_cl.isna().sum()

artist_name    0
track_name     0
song_no        0
dtype: int64

In [ ]:
song_df_cl['song_no'] = song_df_cl.index

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [ ]:
song_df_cl.tail()

,artist_name,track_name,song_no
37774871,"帆足圭吾, 岡部啓一",曖昧ナ希望,37774871
37774880,Jay-Z,Intro/A Million and One Questions/Rhyme No More,37774880
37774885,Moderat,Beatswaysick,37774885
37774908,Raphael,Joy of Forgiveness,37774908
37774923,Joey Anderson,Maidens Response,37774923


In [ ]:
song_df_cl_copy = song_df_cl.copy(deep=True)

In [ ]:
# apply transformations to get data in the correct format
def get_song_id(s):
  """
  Assign index no. of song from song_df to input df
  """
  try:
    return song_df_cl[(song_df_cl['artist_name'].eq(s['artist_name'])) &
                       (song_df_cl['track_name'].eq(s['track_name']))].index.item()
  except IndexError:
    pass

In [ ]:
# make copy of big_df for modifying
big_df_cl = big_plays.copy(deep=True)

In [ ]:
# Try using a dask dataframe for speed purposes
# This didn't really work...
# big_df_cl = dd.from_pandas(big_df_cl, npartitions=10)

In [ ]:
# apply get_song_id to big_df_cl
# apply in a vectorized manner to improve performance
# big_df_cl['song_id'] = song_df_cl[(song_df_cl['artist_name'] == (big_df_cl['artist_name'])) &
#                        (song_df_cl['track_name'] == (big_df_cl['track_name']))].index.item()

In [ ]:
cols = ['artist_name', 'track_name']
big_df_cl = big_df_cl.join(song_df_cl.set_index(cols), on=cols)

In [ ]:
big_df_cl.drop(columns=['artist_name', 'track_name'], inplace=True)

KeyError: ignored

In [ ]:
big_df_cl.isna().sum()

user_name     0
song_no      21
dtype: int64

In [ ]:
big_df_cl.dropna(inplace=True)
big_df_cl.isna().sum()

user_name    0
song_no      0
dtype: int64

In [ ]:
big_df_cl['song_no'] = big_df_cl['song_no'].astype(int)
big_df_cl.head()

,user_name,song_no
0,Skeebadoo,0
1,Skeebadoo,1
2,Skeebadoo,2
3,Skeebadoo,3
4,Svarthjelm,4


In [ ]:
# We'll save the big_df_cl and song_df_cl just in case here
big_df_cl.to_csv('/drive/My Drive/Colab Notebooks/listen_table.csv')
song_df_cl.to_csv('/drive/My Drive/Colab Notebooks/song_table.csv')